# dt = 0.02

In [ ]:
from matplotlib import colors


cdict = {'red':   ((0.0, 0.0, 0.0),
                   (0.5, 1.0, 1.0),
                   (1.0, 1.0, 1.0)),

         'green': ((0.0, 116./255., 116./255.),
                   (0.5, 1., 1.),
                   (1.0, 121./255., 121./255.)),

         'blue':  ((0.0, 122./255., 122./255.),
                   (0.5, 1., 1.0),
                   (1.0, 0.0, 0.0))
        }
       

TUI = colors.LinearSegmentedColormap('TUI', cdict)
norm = colors.Normalize(vmin=0, vmax=1) 

color_hist = TUI(norm(1))#"#ff885a"
color_chi = "black"
color_threshold="black"
color_region = "grey"
color_res = TUI(norm(0))

In [ ]:
from models.lib.firstorderode.lorenz import Lorenz63
from models.lib.firstorderode.roessler import Roessler76
from models.lib.firstorderode.sprott import SprottAttractors


models = [*SprottAttractors[1:4], Roessler76, Lorenz63]

In [ ]:
from scipy.stats import chi2
import numpy as np
import matplotlib.pyplot as plt
from brokenaxes import brokenaxes
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset


dt_02 = True

models_name = ["Sprott B", "Sprott C", "Sprott D", "Roessler76", "Lorenz63"]
data = np.load(f"results/reference_values_ranges{"_02" if dt_02 else ""}.npy")
#data[:, :, -1] = chi2(10).ppf(np.exp(data[:, :, -1]))
data = data.transpose((1, 2, 0))
data.shape


chi2 = chi2(df=10)
quantiles = np.zeros((5, 4, 2))
fontsize=12

print(f"Reference Samples: {data.shape[-1]}")

for i_model, dd in enumerate(data):
    fig, axs = plt.subplots(4, 2, figsize=(10,7), width_ratios=[0.2, 0.8], sharey="row")
    # load reservoir data
    data2 = np.load(f"results/spectral_radius_n_nodes_20/uncoupled{"" if not dt_02 else "_dt_02"}_{models[i_model].__name__}.npz")
    n_linspace = 100
    names = data2["names"][0, 0]
    measures = data2["measures"][0].T
    n_samples = measures.shape[1] // n_linspace
    measures = measures.reshape(measures.shape[0], n_linspace, n_samples)
    name2index = { name:i for i, name in enumerate(names)}
    idx_bnd_osc = np.logical_and(measures[name2index["bounded"]], measures[name2index["ozillating"]])

    print(f"Reservoir samples {models[i_model].__name__} Samples: {idx_bnd_osc.sum()}")
    
    for i_measure, d in reversed(list(enumerate(dd))):
        
        i = i_measure
        ax = axs[i, 1]
        ax_detail = axs[i, 0]

        j = i % 4
        if j == 0:
            fig.suptitle(models_name[i_model], fontsize=fontsize)
        if i >= 20-4-1:
            ax.set_xlabel("Measure",fontsize=fontsize)
            ax_twin_detail.set_xlabel("Measure",fontsize=fontsize)
        
        if i_measure == 0:
            d = 1-d

        heikki_cut_off = 50
        if i_model == 1:
            heikki_cut_off = 100
        elif i_model == 2:
            heikki_cut_off = 200
        elif i_model == 3: 
            heikki_cut_off = 20000

    	# blue ghistogram

        
        name = ["AIncs", "Adev", "Ddev", "Heikki"][i_measure]
        
        if name == "AIncs":
            measure = 1 - measures[name2index["AIncs"]][idx_bnd_osc]
        else:  
            measure = measures[name2index[name]][idx_bnd_osc]

        # calculate quantiles 

        if i_measure == 3:
            qmax = chi2.ppf(0.95)
            qmin = 0
        else:
            qmax = np.quantile(d, 0.95)

        #calc bins

        if  i_measure == 3 and heikki_cut_off is not None:
            _, bins = np.histogram([*measure[measure < heikki_cut_off], * d[d < heikki_cut_off]], bins = 101) #determine bins
        else:
            _, bins = np.histogram([*measure, *d], bins = 101) #determine bins
        
        # plot orange hist

        if i_measure != 0:
            n, axbins, _ = ax.hist(d, bins = 101, label="Reference", color=color_hist, histtype= "stepfilled")#, alpha=0.7)
            ax_detail.hist(d, bins = 101, color=color_hist, histtype= "stepfilled")
            ax_detail.set_ylabel(["AExc", "ADev", "TVar", "GCI"][i] + "\nCounts", fontsize=fontsize)
        else:
            n, axbins, _ = ax.hist(d, bins = bins, label="Reference", color=color_hist, histtype= "stepfilled")#, alpha=0.7)
            ax_detail.hist(d, bins = bins, color=color_hist, histtype= "stepfilled")
            ax_detail.set_ylabel(["AExc", "ADev", "TVar", "GCI"][i] + "\nCounts", fontsize=fontsize)


        ax.tick_params(axis='y', colors=color_hist)
        ax.spines['left'].set_color(color_hist)

        ax_detail.tick_params(axis='y', colors=color_hist)
        ax_detail.spines['left'].set_color(color_hist)

        # plot measure hist
        ax_twin = ax.twinx()
        ax_twin_detail = ax_detail.twinx()
        ax_twin_detail.set_yticklabels([])

        ax_twin.tick_params(axis='y', colors=color_res)
        ax_twin.spines['left'].set_color(color_res)

        ax_twin_detail.tick_params(axis='y', colors=color_res)
        ax_twin_detail.spines['left'].set_color(color_res)

        counts, _, _ = ax_twin.hist(measure, bins = bins, label=f"Reservoir", color=color_res, alpha=0.7, histtype= "stepfilled")
        ax_twin_detail.hist(measure, bins = bins, color=color_res, alpha=0.7, histtype= "stepfilled")
        print(np.sum(counts) / measure.size)

        # threshold and shaded area

        axxlim = ax.get_xlim()

        ax.tick_params(labelbottom=True)
        ax.axvspan(ax.get_xlim()[0], qmax, alpha=0.5, color=color_region, label="Acceptance \nRegion", zorder=0) 
        ax.axvline(qmax, color=color_threshold, ls=(0, (3, 3, 1, 3)), label="Threshold", lw=1)
        
        ax_detail.tick_params(labelbottom=True)
        ax_detail.axvspan(ax.get_xlim()[0], qmax, alpha=0.5, color=color_region, zorder=0) 
        ax_detail.axvline(qmax, color=color_threshold, ls=(0, (3, 3, 1, 3)), lw=1)

        ax.set_xlim(axxlim)

        #chi 2

        if i_measure != 3:
            ax_detail.set_xlim(axxlim[0], qmax + 0.3 * (qmax - axxlim[0]))
        else:
            ax_detail.set_xlim(0, 25)

        if i_measure == 3:
            x = np.linspace(*ax.get_xlim(), 1000)
            ax.plot(x, chi2.pdf(x) * np.sum(n) * (axbins[1] - axbins[0]), color=color_chi, label=r"$\chi^2 (df = 10)$")
            x = np.linspace(*ax_detail.get_xlim(), 1000)
            ax_detail.plot(x, chi2.pdf(x) * np.sum(n) * (axbins[1] - axbins[0]), color=color_chi)

        #plt.setp(ax.xaxis.get_majorticklabels(), rotation=20)

        if i_measure == 3:
            fig.legend( bbox_to_anchor=(1,1), bbox_transform=axs[0,1].transAxes)
            #ax.legend()

        i += 1
    #fig.autofmt_xdate()
    fig.tight_layout()
    fig.subplots_adjust( wspace=0.1)
    plt.savefig(f"pictures/supplemental/quantiles{"_02" if dt_02 else ""}_{models[i_model].__name__}.pdf")
    plt.show()

# dt = 0.1

In [ ]:
from scipy.stats import chi2
import numpy as np
import matplotlib.pyplot as plt
from brokenaxes import brokenaxes
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset


dt_02 = False

models_name = ["Sprott B", "Sprott C", "Sprott D", "Roessler76", "Lorenz63"]
data = np.load(f"results/reference_values_ranges{"_02" if dt_02 else ""}.npy")
#data[:, :, -1] = chi2(10).ppf(np.exp(data[:, :, -1]))
data = data.transpose((1, 2, 0))
data.shape


chi2 = chi2(df=10)
quantiles = np.zeros((5, 4, 2))
fontsize=12

print(f"Reference Samples: {data.shape[-1]}")

for i_model, dd in enumerate(data):
    fig, axs = plt.subplots(4, 2, figsize=(10,7), width_ratios=[0.2, 0.8], sharey="row")
    # load reservoir data
    data2 = np.load(f"results/spectral_radius_n_nodes_20/uncoupled{"" if not dt_02 else "_dt_02"}_{models[i_model].__name__}.npz")
    n_linspace = 100
    names = data2["names"][0, 0]
    measures = data2["measures"][0].T
    n_samples = measures.shape[1] // n_linspace
    measures = measures.reshape(measures.shape[0], n_linspace, n_samples)
    name2index = { name:i for i, name in enumerate(names)}
    idx_bnd_osc = np.logical_and(measures[name2index["bounded"]], measures[name2index["ozillating"]])

    print(f"Reservoir samples {models[i_model].__name__} Samples: {idx_bnd_osc.sum()}")
    
    for i_measure, d in reversed(list(enumerate(dd))):
        
        i = i_measure
        ax = axs[i, 1]
        ax_detail = axs[i, 0]

        j = i % 4
        if j == 0:
            fig.suptitle(models_name[i_model], fontsize=fontsize)
        if i >= 20-4-1:
            ax.set_xlabel("Measure",fontsize=fontsize)
            ax_twin_detail.set_xlabel("Measure",fontsize=fontsize)
        
        if i_measure == 0:
            d = 1-d

        if i_model in (3,):
            heikki_cut_off = 8500
        elif i_model in (1,):
            heikki_cut_off = 10000
        elif i_model in (2,):
            heikki_cut_off = 12000
        elif i_model in (4,):
            heikki_cut_off = 1500
        else:
            heikki_cut_off = 3500

    	# blue ghistogram

        
        name = ["AIncs", "Adev", "Ddev", "Heikki"][i_measure]
        
        if name == "AIncs":
            measure = 1 - measures[name2index["AIncs"]][idx_bnd_osc]
        else:  
            measure = measures[name2index[name]][idx_bnd_osc]

        # calculate quantiles 

        if i_measure == 3:
            qmax = chi2.ppf(0.95)
            qmin = 0
        else:
            qmax = np.quantile(d, 0.95)

        #calc bins

        if  i_measure == 3 and heikki_cut_off is not None:
            _, bins = np.histogram([*measure[measure<heikki_cut_off], * d[d<heikki_cut_off]], bins = 101) #determine bins
            bins = bins[bins < heikki_cut_off]
        else:
            _, bins = np.histogram([*measure, *d], bins = 101) #determine bins
        
        # plot orange hist

        if i_measure not in (0, ):
            n, axbins, _ = ax.hist(d, bins = 101, range=(0, 100) if i_measure == 3 else None, label="Reference", color=color_hist, histtype= "stepfilled")#, alpha=0.7)
            ax_detail.hist(d, bins = 101, range=(0, 100) if i_measure == 3 else None, color=color_hist, histtype= "stepfilled")
            ax_detail.set_ylabel(["AExc", "ADev", "TVar", "GCI"][i] + "\nCounts", fontsize=fontsize)
        else:
            n, axbins, _ = ax.hist(d, bins = bins, label="Reference", color=color_hist, histtype= "stepfilled")#, alpha=0.7)
            ax_detail.hist(d, bins = bins, color=color_hist, histtype= "stepfilled")
            ax_detail.set_ylabel(["AExc", "ADev", "TVar", "GCI"][i] + "\nCounts", fontsize=fontsize)

        

        ax.tick_params(axis='y', colors=color_hist)
        ax.spines['left'].set_color(color_hist)

        ax_detail.tick_params(axis='y', colors=color_hist)
        ax_detail.spines['left'].set_color(color_hist)

        # plot measure hist
        ax_twin = ax.twinx()
        ax_twin_detail = ax_detail.twinx()
        ax_twin_detail.set_yticklabels([])

        ax_twin.tick_params(axis='y', colors=color_res)
        ax_twin.spines['left'].set_color(color_res)

        ax_twin_detail.tick_params(axis='y', colors=color_res)
        ax_twin_detail.spines['left'].set_color(color_res)

        counts, _, _ = ax_twin.hist(measure, bins = bins, label=f"Reservoir", color=color_res, alpha=0.7, histtype= "stepfilled")
        ax_twin_detail.hist(measure, bins = bins, color=color_res, alpha=0.7, histtype= "stepfilled")
        print(np.sum(counts) / measure.size)

        # threshold and shaded area

        axxlim = ax.get_xlim()

        ax.tick_params(labelbottom=True)
        ax.axvspan(ax.get_xlim()[0], qmax, alpha=0.5, color=color_region, label="Acceptance \nRegion", zorder=0) 
        ax.axvline(qmax, color=color_threshold, ls=(0, (3, 3, 1, 3)), label="Threshold", lw=1)
        
        ax_detail.tick_params(labelbottom=True)
        ax_detail.axvspan(ax.get_xlim()[0], qmax, alpha=0.5, color=color_region, zorder=0) 
        ax_detail.axvline(qmax, color=color_threshold, ls=(0, (3, 3, 1, 3)), lw=1)

        ax.set_xlim(axxlim)

        #chi 2

        if i_measure != 3:
            ax_detail.set_xlim(axxlim[0], qmax + 0.3 * (qmax - axxlim[0]))
        else:
            ax_detail.set_xlim(0, 25)

        if i_measure == 3:
            x = np.linspace(*ax.get_xlim(), 1000)
            ax.plot(x, chi2.pdf(x) * np.sum(n) * (axbins[1] - axbins[0]), color=color_chi, label=r"$\chi^2 (df = 10)$")
            x = np.linspace(*ax_detail.get_xlim(), 1000)
            ax_detail.plot(x, chi2.pdf(x) * np.sum(n) * (axbins[1] - axbins[0]), color=color_chi)

        #plt.setp(ax.xaxis.get_majorticklabels(), rotation=20)

        if i_measure == 3:
            fig.legend( bbox_to_anchor=(1,1), bbox_transform=axs[0,1].transAxes)
            #ax.legend()

        i += 1
    #fig.autofmt_xdate()
    fig.tight_layout()
    fig.subplots_adjust( wspace=0.1)
    plt.savefig(f"pictures/supplemental/quantiles{"_02" if dt_02 else ""}_{models[i_model].__name__}.pdf")
    plt.show()

# Single example

In [ ]:
from matplotlib.lines import Line2D
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chi2
import matplotlib.patches as mpatches

fontsize = 13
i_model = 3
fig, axs = plt.subplots(2, 2, figsize=(10, 5.5))
models_name = ["Sprott B", "Sprott C", "Sprott D", "Roessler76", "Lorenz63"]
data = np.load("results/reference_values_ranges_02.npy")
#data[:, :, -1] = chi2(10).ppf(np.exp(data[:, :, -1]))
data = data.transpose((1, 2, 0))[i_model]
data.shape
axs[0]
for j, (ax, measure) in enumerate(zip(axs.flatten()[[2, 3, 1, 0]], data)):
    ax.set_title(["AExc", "ADev", "TVar", "GCI"][j] , fontsize=fontsize)
    ax.set_ylabel("\nCounts", fontsize=fontsize)
    ax.set_xlabel("Measure Value", fontsize=fontsize)
    d = measure

    # if j == 0:
    #     ax.set_xlim(-0.005, 0.025)

    # if j == 1:
    #     ax.set_xlim(111_500, 112_500)

    # if j == 2:
    #     ax.set_xlim(0.865, 0.915)

    
    if j == 0:
        d = 1 - d

    qmin = np.min(d)

    if j == 3:
        qmax = chi2.ppf(0.95, df=10)
        d = d[d < 50]
        ax.set_xlim(-1, 35)

    else:
        qmax = np.quantile(d, 0.95)

    n, bins, _ = ax.hist(d, bins = 101, label="Reference", color=color_hist, histtype= "stepfilled")

    if j == 3:
        x = np.linspace(-1, 35, 1000)
        ax.plot(x, chi2.pdf(x, df=10)*sum(n)*(bins[1]-bins[0]), color=color_chi, label=r"$\chi^2(df = 10)$", alpha=0.5)
    ax.axvline(qmax, color=color_threshold, ls=(0, (3, 3, 1, 3)), label="Threshold")

    xlim = ax.get_xlim()
    ax.axvspan(xlim[0], qmax, alpha=0.5, color=color_region, label="acceptance region", zorder=0) 
    ax.set_xlim(xlim)
    
    if j == 0:
        # where some data has already been plotted to ax
        handles, labels = ax.get_legend_handles_labels()

        # handles is a list, so append manual patch
        handles.append(Line2D([0], [0], label='$\chi^2(df = 10)$', color='black')) 

        # plot the legend
        fig.legend(handles=handles, loc='upper right', facecolor='white', framealpha=1,  bbox_to_anchor=[1., 1.0], bbox_transform=axs[0, 1].transAxes)
        ax.set_zorder(100)

    
        
plt.tight_layout()
plt.subplots_adjust(hspace=0.5)  # Increase this value for more space

plt.savefig("pictures/quantile_02_single.pdf")